In [ ]:
!pip install pandas numpy scapy scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score

# Load Data and Create Synthetic Labels
def load_and_label_data(file_path):
    try:
        df = pd.read_csv(file_path)

        # Handle potential infinite and NaN values initially
        df.replace([np.inf, -np.inf], np.nan, inplace=True)

        # Encode 'Label' column if it exists, otherwise create synthetic labels
        if 'Label' in df.columns:
            le = LabelEncoder()
            df['Label'] = le.fit_transform(df['Label'])
        else:
            df['Synthetic_Label'] = (df['Total Length of Fwd Packets'] > 500).astype(int)

    except Exception as e:
        print("Error reading file:", e)
        return None

    return df

# Preprocess Data
def preprocess_data(df):

    drop_columns = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp']
    df = df.drop(columns=[col for col in drop_columns if col in df.columns], errors='ignore')


    label_col = 'Label' if 'Label' in df.columns else 'Synthetic_Label'
    non_numeric_cols = df.select_dtypes(include=['object']).columns.tolist()
    df = df.drop(columns=non_numeric_cols, errors='ignore')

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)  # Example: Fill NaN with column means

    X = df.drop(columns=[label_col]).values
    y = df[label_col].values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

# Define Simple Neural Network Model
class IntrusionDetector(nn.Module):
    def __init__(self, input_size, num_classes):
        super(IntrusionDetector, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def main():
    # Load and label the dataset
    file_path = '/root/.cache/kagglehub/datasets/chethuhn/network-intrusion-dataset/versions/1/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'  # Path to your dataset
    df = load_and_label_data(file_path)
    if df is not None:
        X, y = preprocess_data(df)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

        # Convert to PyTorch tensors
        train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train))
        test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test))

        train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
        test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

        # Initialize model, criterion, and optimizer
        input_size = X.shape[1]
        num_classes = len(np.unique(y))
        model = IntrusionDetector(input_size, num_classes)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # Train the model
        model.train()
        num_epochs = 20
        for epoch in range(num_epochs):
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

        # Evaluate the model
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                all_preds.extend(predicted.numpy())
                all_labels.extend(labels.numpy())

        # Calculate and output accuracy and precision
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='macro')

        print(f'Accuracy: {accuracy:.4f}')
        print(f'Precision: {precision:.4f}')

        # Classification report
        print(classification_report(all_labels, all_preds))

if __name__ == "__main__":
    main()

Epoch [1/20], Loss: 0.0140
Epoch [2/20], Loss: 0.0040
Epoch [3/20], Loss: 0.0032
Epoch [4/20], Loss: 0.0029
Epoch [5/20], Loss: 0.0025
Epoch [6/20], Loss: 0.0022
Epoch [7/20], Loss: 0.0022
Epoch [8/20], Loss: 0.0023
Epoch [9/20], Loss: 0.0020
Epoch [10/20], Loss: 0.0019
Epoch [11/20], Loss: 0.0016
Epoch [12/20], Loss: 0.0018
Epoch [13/20], Loss: 0.0017
Epoch [14/20], Loss: 0.0018
Epoch [15/20], Loss: 0.0017
Epoch [16/20], Loss: 0.0014
Epoch [17/20], Loss: 0.0014
Epoch [18/20], Loss: 0.0017
Epoch [19/20], Loss: 0.0015
Epoch [20/20], Loss: 0.0014
Accuracy: 0.9993
Precision: 0.9983
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39251
           1       1.00      1.00      1.00      5898

    accuracy                           1.00     45149
   macro avg       1.00      1.00      1.00     45149
weighted avg       1.00      1.00      1.00     45149



In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_score

# Load Data and Create Synthetic Labels
def load_and_label_data(file_path, num_rows=19000):
    try:
        df = pd.read_csv(file_path, nrows=num_rows)

        if 'Label' in df.columns:
            le = LabelEncoder()
            df['Label'] = le.fit_transform(df['Label'].fillna('Unknown'))

        if 'Label' not in df:
            df['Synthetic_Label'] = (df['Total Length of Fwd Packets'] > 500).astype(int)

    except Exception as e:
        print("Error reading file:", e)
        return None

    return df

# Preprocess Data
def preprocess_data(df):
    drop_columns = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Timestamp']
    df = df.drop(columns=[col for col in drop_columns if col in df.columns], errors='ignore')

    label_col = 'Label' if 'Label' in df.columns else 'Synthetic_Label'

    non_numeric_cols = df.select_dtypes(include=['object']).columns.tolist()
    df = df.drop(columns=non_numeric_cols, errors='ignore')

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.mean(), inplace=True)

    X = df.drop(columns=[label_col]).values
    y = df[label_col].values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

# Define Simple Neural Network Model
class IntrusionDetector(nn.Module):
    def __init__(self, input_size, num_classes):
        super(IntrusionDetector, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def evaluate_model(X, y, num_splits=5):
    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    results = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        train_data = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train))
        test_data = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test))

        train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
        test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

        model = IntrusionDetector(input_size=X.shape[1], num_classes=len(np.unique(y)))
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        model.train()
        for epoch in range(10):
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f'Epoch [{epoch+1}/10], Loss: {running_loss/len(train_loader):.4f}')

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                all_preds.extend(predicted.numpy())
                all_labels.extend(labels.numpy())

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='macro')

        print(f'Fold Accuracy: {accuracy:.4f}')
        print(f'Fold Precision: {precision:.4f}')
        results.append((accuracy, precision))

    mean_accuracy, mean_precision = np.mean(results, axis=0)
    print(f'Mean Accuracy: {mean_accuracy:.4f}, Mean Precision: {mean_precision:.4f}')

def main():
    file_path = '/root/.cache/kagglehub/datasets/chethuhn/network-intrusion-dataset/versions/1/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
    df = load_and_label_data(file_path, num_rows=1000)

    if df is not None:
        X, y = preprocess_data(df)
        evaluate_model(X, y)

if __name__ == "__main__":
    main()

Epoch [1/10], Loss: 0.5727
Epoch [2/10], Loss: 0.2615
Epoch [3/10], Loss: 0.0861
Epoch [4/10], Loss: 0.0469
Epoch [5/10], Loss: 0.0341
Epoch [6/10], Loss: 0.0326
Epoch [7/10], Loss: 0.0255
Epoch [8/10], Loss: 0.0229
Epoch [9/10], Loss: 0.0215
Epoch [10/10], Loss: 0.0187
Fold Accuracy: 0.9950
Fold Precision: 0.9926
Epoch [1/10], Loss: 0.5695
Epoch [2/10], Loss: 0.2363
Epoch [3/10], Loss: 0.0791
Epoch [4/10], Loss: 0.0449
Epoch [5/10], Loss: 0.0371
Epoch [6/10], Loss: 0.0300
Epoch [7/10], Loss: 0.0270
Epoch [8/10], Loss: 0.0242
Epoch [9/10], Loss: 0.0223
Epoch [10/10], Loss: 0.0217
Fold Accuracy: 0.9950
Fold Precision: 0.9963
Epoch [1/10], Loss: 0.5575
Epoch [2/10], Loss: 0.2407
Epoch [3/10], Loss: 0.0845
Epoch [4/10], Loss: 0.0462
Epoch [5/10], Loss: 0.0378
Epoch [6/10], Loss: 0.0321
Epoch [7/10], Loss: 0.0299
Epoch [8/10], Loss: 0.0255
Epoch [9/10], Loss: 0.0250
Epoch [10/10], Loss: 0.0207
Fold Accuracy: 0.9900
Fold Precision: 0.9857
Epoch [1/10], Loss: 0.5480
Epoch [2/10], Loss: 0.203